In [65]:
import pandas as pd
import copy

In [66]:
MasterCodebook = pd.DataFrame(
    {
        "Variable Name": [],
        "Label": [],
        "Alternative Labels": [],
        "Type": [],
        "Data Type": [],
        "Value Codes": [],
        "Missing Code": []
    }
)

In [67]:
UltraData = pd.DataFrame(
 {
     "DbID": [],
     "Studiengang": [],
     "Fakultät": [],
     "Geschlecht": [],
     "Jahr": [],
     "Semester": [],
     "Dozent": [],
     "Fachbereich": [],
     "Modulinfo": []
 }   
)

In [68]:
#key = Variablenname aus exportierten Datensatz ("alt")
#value = Variablenname aus *MasterCodebook* ("neu")

variablennamenumwandler = {}

In [69]:
exportdata = pd.read_excel(io="/Users/Vale-chan/Documents/ArtologikExports/HeS20_Sek I MNGW_Robin_10068 9. Semester_Evolutionsbiologie der Organismen k.nt.fwd.1.9.xlsx", sheet_name=None, engine="openpyxl")


In [70]:
for _,zeile in exportdata["VariableView"].iterrows():

    exportlabel = zeile["Label"]

    found = False

    for ii,xx in enumerate(MasterCodebook["Alternative Labels"]):
        if exportlabel in xx:
            variablennamenumwandler[zeile["Variable Name"]] = MasterCodebook["Variable Name"].iloc[ii]
            found = True
            break

    if not found:
        # TODO
        # innehaue, dasses frogt, ob meh das als alternatives lable will
        z = copy.deepcopy(zeile)
        z["Alternative Labels"] = [zeile["Label"]]
        varnameneu = "VAR" + str((MasterCodebook.shape[0]+1))
        variablennamenumwandler[z["Variable Name"]] = varnameneu
        z["Variable Name"] = varnameneu
        MasterCodebook = MasterCodebook.append(z)

In [71]:
MasterCodebook

,Variable Name,Label,Alternative Labels,Type,Data Type,Value Codes,Missing Code
0,VAR1,Die Dozentin / der Dozent bezieht das Vorwiss...,[ Die Dozentin / der Dozent bezieht das Vorwis...,Single-Choice - Radio Buttons,Numeric,1 = stimmt gar nicht\n2 = stimmt eher nicht\n3...,999.0
1,VAR2,Die Dozentin / der Dozent gestaltet - dem Lern...,[Die Dozentin / der Dozent gestaltet - dem Ler...,Single-Choice - Radio Buttons,Numeric,1 = stimmt gar nicht\n2 = stimmt eher nicht\n3...,999.0
2,VAR3,Bei Bedürfnissen und Notwendigkeiten seitens d...,[Bei Bedürfnissen und Notwendigkeiten seitens ...,Single-Choice - Radio Buttons,Numeric,1 = stimmt gar nicht\n2 = stimmt eher nicht\n3...,999.0
3,VAR4,Die Dozentin / der Dozent stellt Bezüge zwisch...,[Die Dozentin / der Dozent stellt Bezüge zwisc...,Single-Choice - Radio Buttons,Numeric,1 = stimmt gar nicht\n2 = stimmt eher nicht\n3...,999.0
4,VAR5,Auf Fragen von Studierenden geht die Dozentin ...,[Auf Fragen von Studierenden geht die Dozentin...,Single-Choice - Radio Buttons,Numeric,1 = stimmt gar nicht\n2 = stimmt eher nicht\n3...,999.0
5,VAR6,Die Dozentin / der Dozent stellt Bezüge zur Zi...,[Die Dozentin / der Dozent stellt Bezüge zur Z...,Single-Choice - Radio Buttons,Numeric,1 = stimmt gar nicht\n2 = stimmt eher nicht\n3...,999.0
6,VAR7,Die Dozentin / der Dozent stellt Bezüge zur Fa...,[Die Dozentin / der Dozent stellt Bezüge zur F...,Single-Choice - Radio Buttons,Numeric,1 = stimmt gar nicht\n2 = stimmt eher nicht\n3...,999.0
7,VAR8,"Der Dozentin / dem Dozenten gelingt es, eine ...","[ Der Dozentin / dem Dozenten gelingt es, eine...",Single-Choice - Radio Buttons,Numeric,1 = stimmt gar nicht\n2 = stimmt eher nicht\n3...,999.0
8,VAR9,"Der Dozentin / dem Dozenten gelingt es, Theor...","[ Der Dozentin / dem Dozenten gelingt es, Theo...",Single-Choice - Radio Buttons,Numeric,1 = stimmt gar nicht\n2 = stimmt eher nicht\n3...,999.0
9,VAR10,Die Dozentin / der Dozent gibt lernförderndes ...,[Die Dozentin / der Dozent gibt lernförderndes...,Single-Choice - Radio Buttons,Numeric,1 = stimmt gar nicht\n2 = stimmt eher nicht\n3...,999.0


In [72]:
variablennamenumwandler

{'1.2': 'VAR1',
 '1.4': 'VAR2',
 '1.5': 'VAR3',
 '2.2': 'VAR4',
 '2.4': 'VAR5',
 '2.5': 'VAR6',
 '2.7': 'VAR7',
 '2.8': 'VAR8',
 '2.9': 'VAR9',
 '3.3': 'VAR10',
 '5.3': 'VAR11',
 '7.9': 'VAR12',
 '8.1': 'VAR13',
 '8.2': 'VAR14',
 '8.3': 'VAR15',
 '8.4': 'VAR16',
 '8.5': 'VAR17',
 '8.6': 'VAR18',
 '10.1': 'VAR19',
 '12.1': 'VAR20',
 '12.2': 'VAR21',
 '12.5': 'VAR22',
 'Was würden Sie an der Lehrveranstaltung ändern? Warum?': 'VAR23',
 '13.7 freiwillig Name:': 'VAR24'}